<a href="https://colab.research.google.com/github/orbenishay/Classification/blob/master/number%20of%20shares%20in%20social%20networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Number of shares in social networks**

https://code.datasciencedojo.com/datasciencedojo/datasets/tree/master/Online%20News%20Popularity

In [1]:
# import modules
# General tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import warnings

# Pandas
from datetime import datetime
from datetime import timedelta



# Pipieline summarized:
from scipy.optimize import curve_fit# For curve fitting
from sklearn.base import TransformerMixin # For transformations and predictions
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator # For predictions
from sklearn.metrics import mean_squared_log_error as msle # For scoring
from sklearn.metrics import mean_squared_error as mse # For scoring
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances
import pydot as pyd # For the tree visualization
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.model_selection import train_test_split # For validation
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix, accuracy_score # For classification
from sklearn.metrics import precision_score, recall_score, f1_score # For classification
from sklearn.metrics import log_loss # For classification

!pip install pydot

import sys

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


### **Load files (dataset and dictionary)**

In [7]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import files
    uploaded = files.upload()

Saving Dictionary.csv to Dictionary (1).csv


### **Create dadtaframe**

In [10]:
shares=pd.read_csv('Shares.csv', index_col='url')
shares.head()

dictionary=pd.read_csv('Dictionary.csv', index_col='Column Position')
dictionary.head()

,Atrribute Name,Definition,Data Type,Example
Column Position,,,,
1,URL,URL Of The Article (Non-Predictive),Qualitative,"""http://mashable.com/2013/01/07/amazon-instant..."
2,Timedelta,Days Between The Article Publication And The ...,Quantitative,731
3,N_Tokens_Title,Number Of Words In The Title,Quantitative,12
4,N_Tokens_Content,Number Of Words In The Content,Quantitative,219
5,N_Unique_Tokens,Rate Of Unique Words In The Content,Quantitative,0.663594467


### **Data Cleansing**

In [12]:
shares = shares[shares.isnull().sum(axis=1)<3] # Remove rows with more than 3 `NaN`'s.
shares = shares.loc[:, ~shares.columns.str.contains(r'review', case=False)] # remove specific columns
shares.drop_duplicates(inplace=True) # Remove duplicates
shares.drop_duplicates(subset=['director_name', 'title_year'], inplace=True) # Remove duplicates with subsets: Only consider certain columns for identifying duplicates, by default use all of the columns
shares.index.value_counts().head() # look for duplications

# Clean the data:
shares = shares.loc[shares['bedrooms']<8,:] # filter dataset based on a column's values
shares = shares.loc[(shares.bedrooms<8) & (shares.bathrooms<8)]
shares = shares.loc[shares.accommodates * shares.minimum_nights != 0] # remove rows with 0 values at either columns
shares = shares.loc[(shares.bedrooms - shares.bathrooms).abs() < 5] # remove based on TPA

# def drop_zeros(df): # remove zeros
#     return df.loc[df.x * df.y * df.z != 0]
# zeros_dropper = FunctionTransformer(drop_zeros, validate=False)
# diamonds = zeros_dropper.fit_transform(diamonds)

shares.shape # after each cleaning step: check how many rows were deleted 

(39644, 60)

In [0]:
# More data cleansing
for col in bnb.columns: # remove outliers >3 sigma from all the fields
    if bnb[col].dtype == 'float64':
        std = bnb[col].std()
        ave = bnb[col].mean()
        bnb = bnb.loc[bnb[col].between(ave-3*std, ave+3*std)]
        print(f'processing {col:10} --> {bnb.shape[0]:5} assets remain')

# xz_model = LinearRegression().fit(diamonds[['x']], diamonds.z) # remove bivariate outliers
# z_pred = xz_model.predict(diamonds[['x']])
# diamonds = diamonds.loc[(diamonds.z-z_pred).abs()<0.35]

# x_carat_model = LinearRegression().fit(diamonds[['x']], diamonds.carat) # remove by linear regression
# carat_pred = x_carat_model.predict(diamonds[['x']])
# diamonds = diamonds.loc[(diamonds.carat-carat_pred).abs()<0.3]
# sns.scatterplot(x='x', y='carat', data=diamonds)

# def exponential(x, a, b, c, d): # remove by exponential regression
#     return a + b*np.exp(c*x+d)
# popt, _ = curve_fit(exponential, xdata=diamonds.x, ydata=diamonds.carat)
# print(popt)
# a, b, c, d = popt
# carat_pred = diamonds.x.apply(exponential, a=a, b=b, c=c, d=d) # plot the curve
# plt.plot(diamonds.x, diamonds.carat, '.b', 
#               diamonds.x, carat_pred, '.g')
# plt.text(6, 0.5, r'$carat = -0.47+19.56 x exp(0.3x-4.56)$')
# residuals = (diamonds.carat - carat_pred).abs() #evaluate the residuals and remove data points as we wish.
# residuals.hist(bins=100)

### **Dataset explration**

In [18]:
shares.describe() # describe with subset
shares[['room_type','price']].groupby('room_type').describe() # describe with subset
shares.nunique() # how many values per field
shares.loc[:,['d_centre','price']] # slice dataframe (show specific columns)
shares.timedelta.loc[:,].count() # count of entries in a column of a df
shares.shape # dimention of df

shares.bedrooms.value_counts().head(10) #Count entries per value:


,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,shares
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,354.530471,10.398749,546.514731,0.548216,0.996469,0.689175,10.883690,3.293638,4.544143,1.249874,4.548239,7.223767,0.052946,0.178009,0.157855,0.058597,0.185299,0.212567,26.106801,1153.951682,312.366967,752324.066694,259277.682421,1117.146610,5657.211151,3135.858639,3998.755396,10329.212662,6401.697580,0.168020,0.186409,0.187544,0.183306,0.143805,0.061876,0.069039,0.130915,0.184599,0.141256,0.216321,0.223770,0.234029,0.443370,0.119309,0.039625,0.016612,0.682150,0.287934,0.353825,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,3395.380184
std,214.163767,2.114037,471.107508,3.520708,5.231231,3.264816,11.332017,3.855141,8.309434,4.107855,0.844406,1.909130,0.223929,0.382525,0.364610,0.234871,0.388545,0.409129,69.633215,3857.990877,620.783887,214502.129573,135107.757337,1137.456951,6098.871957,1318.150397,19738.670516,41027.576613,24211.332231,0.373889,0.389441,0.390353,0.386922,0.350896,0.240933,0.253524,0.337312,0.262975,0.219707,0.282145,0.295191,0.289183,0.116685,0.096931,0.017429,0.010828,0.190206,0.156156,0.104542,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,11626.950749
min,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.393750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,1.000000
25%,164.000000,9.000000,246.000000,0.470870,1.000000,0.625739,4.000000,1.000000,1.000000,0.000000,4.478404,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,445.000000,141.750000,843300.000000,172846.875000,0.000000,3562.101631,2382.448565,639.000000,1100.000000,981.187500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025051,0.025012,0.028571,0.028571,0.028574,0.396167,0.057757,0.028384,0.009615,0.600000,0.185185,0.306244,0.050000,0.600000,-0.328383,-0.700000,-0.125000,0.000000,0.000000,0.